# # 모듈 및 함수
---

In [1]:
import pandas as pd
import numpy as np
import os
import open3d as o3d
import pickle as pkl
import re
import json
import shutil
import glob
from collections import Counter
from tqdm import tqdm
from sklearn.model_selection import train_test_split


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

def convert_x1y1x2y2_to_tlwh(bbox):
    '''
    :param bbox: x1 y1 x2 y2
    :return: tlwh: top_left x   top_left y    width   height
    '''
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    return np.array(([bbox[0], bbox[1], w, h]))

# rotation matrix
def roty(t, Rx=90/180*np.pi):
    ''' Rotation about the y-axis. '''
    c = np.cos(t)
    s = np.sin(t)
    
    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)

def xyz2xyxy(x, y, z, l, w, h, rot_y, extrinsic, intrinsic):
    R = roty(rot_y)
   
    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
    y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];
    
    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
    corners_3d[0, :] = corners_3d[0, :] + x  # x
    corners_3d[1, :] = corners_3d[1, :] + y  # y
    corners_3d[2, :] = corners_3d[2, :] + z  # z
    corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])
    
    point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
    pointx = np.around(point2d/point2d[2])[0]
    pointy = np.around(point2d/point2d[2])[1]

    return min(pointx), min(pointy), max(pointx), max(pointy)

# 문자열 숫자리스트로 바꾸는 함수
def str2list(txt):
    txt = txt.replace('\n', '').split(',')
    txt = list(map(float, txt))
    
    return txt

# 리스트를 문자열로 바꾸는 함수
def list2str(list):
    list = ' '.join(map(str, list))
    
    return list


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# # 데이터 프레임 만들기
---

In [3]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final'


# Z축 이동을 위해서 calib와 매칭하여 이동범위 지정
calib_ls = []
# scenes = []
calibs = sorted(glob.glob(f'{src}/*/calib/camera/camera_0.json'))
for calib in calibs:
    scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
    with open(calib, 'r') as f:
        calib = json.load(f)
    if calib['extrinsic'] not in calib_ls:
        calib_ls.append(calib['extrinsic'])
        # scenes.append(scene)    
calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
             'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
             'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
             'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}


# 라벨데이터로 데이터프레임 생성
dp_ls = []
labels = sorted(glob.glob(f'{src}/*/label/*.json'))
for j, label in enumerate(tqdm(labels)):
    scene = re.findall('\w+', label)[-4]
    frame = re.findall('\w+', label)[-2]

    # calib값 조정
    with open(f'{src}/{scene}/calib/camera/camera_0.json', 'r') as f:
        calib = json.load(f)

    extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
    intrinsic = np.zeros([3, 4])
    intrinsic[:3, :3] = np.asarray(calib['intrinsic']).reshape(3, 3)
    
    for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
        if calib['extrinsic'] == calib_typ[typ]['calib']:
            # extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
            extrinsic[:3, 3] -= extrinsic[:3, 2] * calib_typ[typ]['mov_zpoint']   

    # 컬럼 구성
    globals()[f'dp{j}'] = pd.DataFrame()
    with open(label, 'r') as f:
        json_data = json.load(f)

    for i in range(len(json_data)):
        try:
            id_ = json_data[i]['obj_id']
            class_ = json_data[i]['obj_type']
            psr = json_data[i]['psr']
            point_x, point_y, point_z = psr['position']['x'], psr['position']['y'], psr['position']['z']
            # z값 범위를 줄이기 위해 조정
            for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
                if calib['extrinsic'] == calib_typ[typ]['calib']:
                    mov_point_z = calib_typ[typ]['mov_zpoint']
                    point_z += mov_point_z
            l, w, h = psr['scale']['x'], psr['scale']['y'], psr['scale']['z']
            rot_y = psr['rotation']['z']
            min_x, min_y, max_x, max_y = xyz2xyxy(point_x, point_y, point_z, l, w, h, rot_y, extrinsic, intrinsic)

            data = [
                id_, class_, 
                min_x, min_y, max_x, max_y,
                point_x, point_y, point_z,
                l, w, h,
                rot_y,
                intrinsic.flatten().tolist(), extrinsic.flatten().tolist(), mov_point_z,
                scene, frame
                ]
            
            columns = [
                'id', 'class',
                'min_x', 'min_y', 'max_x', 'max_y',
                'point_x', 'point_y', 'point_z',
                'l', 'w', 'h',
                'rot_y',
                'intrinsic', 'extrinsic', 'mov_point_z',
                'scene', 'frame'
                ]

            frame_data = pd.DataFrame(data=[data], columns=columns)
            globals()[f'dp{j}'] = pd.concat([globals()[f'dp{j}'], frame_data], axis=0)
        except:
            continue
    dp_ls.append(globals()[f'dp{j}'])
    
dp = pd.concat(dp_ls).reset_index(drop=True)

# dp.loc[dp['min_x'] < 0, 'min_x'] = 0
# dp.loc[dp['min_x'] > 1920, 'min_x'] = 1920
# dp.loc[dp['max_x'] < 0, 'max_x'] = 0
# dp.loc[dp['max_x'] > 1920, 'max_x'] = 1920
# dp.loc[dp['min_y'] < 0, 'min_y'] = 0
# dp.loc[dp['min_y'] > 1200, 'min_y'] = 1200
# dp.loc[dp['max_y'] < 0, 'max_y'] = 0
# dp.loc[dp['max_y'] > 1200, 'max_y'] = 1200
dp['id'] = dp['id'].apply(pd.to_numeric, errors='coerce')
dp = dp.dropna(axis=0)
# drop_index = dp.loc[(dp['class']==0)| (dp['class']=='Unknown') | (dp['id'].isnull()) | 
#                     (dp['min_x']-dp['max_x']==0) | (dp['min_y']-dp['max_y']==0) |
#                     (dp['point_y']>0) | (dp['point_y']<-50) | (dp['point_x']<-50) | (dp['point_x']>50)].index
# dp = dp.drop(drop_index).reset_index(drop=True)
dp['id'] = dp['id'].astype(int)
dp.to_csv(f'{dst}/data_info.csv')
# dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info.csv', index_col=0, dtype={'frame':object})

100%|██████████| 58160/58160 [17:41<00:00, 54.80it/s] 


In [ ]:
# 라벨 체크
no_label = []
src = '/data/NIA50/50-2/data/nia50_final/raw'
labels = sorted(glob.glob(f'{src}/*/label/*.json'))
for j, label in enumerate(tqdm(labels)):
    scene = re.findall('\w+', label)[-4]
    frame = re.findall('\w+', label)[-2]

    with open(label, 'r') as f:
        json_data = json.load(f)

    if len(json_data)==0:
        no_label.append(scene)

from collections import Counter

for i in dict(Counter(no_label)).items():
    print(i)

In [ ]:
from collections import Counter

for i in dict(Counter(no_label)).items():
    print(i)

In [ ]:
# l, w, h 범위 확인
np.around(dp.groupby('class').quantile(0.99)[['l', 'w', 'h']].values, 2)

## # 데이터 프레임 불러오기
---

In [3]:
dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info.csv', index_col=0, dtype={'frame':object})
dp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 584991 entries, 0 to 585023
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           584991 non-null  int64  
 1   class        584991 non-null  object 
 2   min_x        584991 non-null  float64
 3   min_y        584991 non-null  float64
 4   max_x        584991 non-null  float64
 5   max_y        584991 non-null  float64
 6   point_x      584991 non-null  float64
 7   point_y      584991 non-null  float64
 8   point_z      584991 non-null  float64
 9   l            584991 non-null  float64
 10  w            584991 non-null  float64
 11  h            584991 non-null  float64
 12  rot_y        584991 non-null  float64
 13  intrinsic    584991 non-null  object 
 14  extrinsic    584991 non-null  object 
 15  mov_point_z  584991 non-null  int64  
 16  scene        584991 non-null  object 
 17  frame        584991 non-null  object 
dtypes: float64(11), int64(2)

## # class 통합 1
---
- Small_Car, Light_Car, Car -> Car
- SUV -> SUV_&_Van
- Van -> SUV_&_Van
- Small_Truck, Medium_Truck, Large_Truck -> Truck
- Mini_Bus, Bus -> Bus
- Special_Vehicle -> 앵커 기준으로 SUV_&_Van, Truck, Special_Vehicle로 나누기
- Two_Wheeler -> Two_Wheeler
- Kickboard, Adult, Kid -> Person

In [22]:
dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info1.csv', index_col=0, dtype={'frame':object})

# class 통합
dp.loc[(dp['class']=='Car') | (dp['class']=='Light_Car') | (dp['class']=='Small_Car'), 'class'] = 'Car'
dp.loc[(dp['class']=='SUV') | (dp['class']=='Van'), 'class'] = 'SUV_&_Van'
dp.loc[(dp['class']=='Adult') | (dp['class']=='Kid') | (dp['class']=='Kickboard'), 'class'] = 'Person'
dp.loc[(dp['class']=='Small_Truck') | (dp['class']=='Medium_Truck') | (dp['class']=='Large_Truck'), 'class'] = 'Truck'
dp.loc[(dp['class']=='Mini_Bus') | (dp['class']=='Bus'), 'class'] = 'Bus'

qt = dp[['class', 'l', 'w', 'h']].groupby('class').quantile(0.99)
dp.loc[(dp['class']=='Special_Vehicle') & (dp['l']<=qt.loc['SUV_&_Van']['l']) & (dp['w']<=qt.loc['SUV_&_Van']['w']) & (dp['h']<=qt.loc['SUV_&_Van']['h']), 'class'] = 'SUV_&_Van'
dp.loc[(dp['class']=='Special_Vehicle') & (dp['l']>qt.loc['SUV_&_Van']['l']) & (dp['w']>qt.loc['SUV_&_Van']['w']) & (dp['h']>qt.loc['SUV_&_Van']['h'])
        & (dp['l']<=qt.loc['Truck']['l']) & (dp['w']<=qt.loc['Truck']['w']) & (dp['h']<=qt.loc['Truck']['h']), 'class'] = 'Truck'

## # class 통합 2
---
- Small_Car, Light_Car, Car -> Car
- SUV -> SUV_&_Van
- Van -> SUV_&_Van
- Small_Truck, Medium_Truck, Large_Truck -> Truck
- Mini_Bus, Bus -> Bus
- Special_Vehicle -> 육안으로 확인하여 Car, SUV_&_VAN, Truck, Special_Vehicle로 나누기 (Special_Vehicle은 지게차, 포크레인, 야쿠르트 아주머니)
- Two_Wheeler -> Two_Wheeler
- Kickboard, Adult, Kid -> Person
- l, w, h가 너무 작거나 큰 객체는 라벨에서 삭제

In [2]:
dp = pd.read_csv('/data/NIA50/50-2/data/nia50_final/data_info.csv', index_col=0, dtype={'frame':object})

# class 통합
dp.loc[(dp['class']=='Car') | (dp['class']=='Light_Car') | (dp['class']=='Small_Car'), 'class'] = 'Car'
dp.loc[(dp['class']=='SUV') | (dp['class']=='Van'), 'class'] = 'SUV_&_Van'
dp.loc[(dp['class']=='Adult') | (dp['class']=='Kid') | (dp['class']=='Kickboard'), 'class'] = 'Person'
dp.loc[(dp['class']=='Small_Truck') | (dp['class']=='Medium_Truck') | (dp['class']=='Large_Truck'), 'class'] = 'Truck'
dp.loc[(dp['class']=='Mini_Bus') | (dp['class']=='Bus'), 'class'] = 'Bus'

cng_sv = pd.read_csv('/data/NIA50/50-2/data/nia50_final/change_special_vehicle.txt', names=['id', 'scene', 'class', 'sub_class'])
for id_, scene in zip(cng_sv['id'].values, cng_sv['scene'].values):
    cng_cls = cng_sv.loc[(cng_sv['id']==id_) & (cng_sv['scene']==scene), 'class']
    dp.loc[(dp['id']==id_) & (dp['scene']==scene) & (dp['class']=='Special_Vehicle'), 'class'] = cng_cls

# 탐지 범위 제한
dp.loc[dp['min_x'] < 0, 'min_x'] = 0
dp.loc[dp['min_x'] > 1920, 'min_x'] = 1920
dp.loc[dp['max_x'] < 0, 'max_x'] = 0
dp.loc[dp['max_x'] > 1920, 'max_x'] = 1920
dp.loc[dp['min_y'] < 0, 'min_y'] = 0
dp.loc[dp['min_y'] > 1200, 'min_y'] = 1200
dp.loc[dp['max_y'] < 0, 'max_y'] = 0
dp.loc[dp['max_y'] > 1200, 'max_y'] = 1200
drop_index = dp.loc[(dp['class']==0)| (dp['class']=='Unknown') | (dp['id'].isnull()) | 
                    (dp['min_x']-dp['max_x']==0) | (dp['min_y']-dp['max_y']==0) |
                    (dp['point_y']>0) | (dp['point_y']<-50) | (dp['point_x']<-50) | (dp['point_x']>50)].index
dp = dp.drop(drop_index).reset_index(drop=True)

# 극단값 제거
for class_ in dp['class'].unique():
    min_Q = dp.loc[dp['class']==class_][['l', 'w', 'h']].quantile(0.1)
    max_Q = dp.loc[dp['class']==class_][['l', 'w', 'h']].quantile(0.99)
    drop_index = dp.loc[(dp['class']==class_) & 
                        ((dp['l']<min_Q[0]) | (dp['w']<min_Q[1]) | (dp['h']<min_Q[2]) | (dp['l']>max_Q[0]) | (dp['w']>max_Q[1]) | (dp['h']>max_Q[2]))].index
    dp = dp.drop(drop_index)

dp = dp.reset_index(drop=True)
dp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424049 entries, 0 to 424048
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           424049 non-null  int64  
 1   class        424049 non-null  object 
 2   min_x        424049 non-null  float64
 3   min_y        424049 non-null  float64
 4   max_x        424049 non-null  float64
 5   max_y        424049 non-null  float64
 6   point_x      424049 non-null  float64
 7   point_y      424049 non-null  float64
 8   point_z      424049 non-null  float64
 9   l            424049 non-null  float64
 10  w            424049 non-null  float64
 11  h            424049 non-null  float64
 12  rot_y        424049 non-null  float64
 13  intrinsic    424049 non-null  object 
 14  extrinsic    424049 non-null  object 
 15  mov_point_z  424049 non-null  int64  
 16  scene        424049 non-null  object 
 17  frame        424049 non-null  object 
dtypes: float64(11), int64(2)

In [153]:
# frame 모자란 scene 확인
check = []
for scene in dp['scene'].unique():
    frame = dp.loc[dp['scene']==scene, 'frame']
    frame_num = len(set(frame))
    if frame_num < 10:
        check.append({'scene': scene, 'frame':set(frame)})

In [ ]:
# # 극단값 제거 (IQR 사용)
# # dp_Q = dp.groupby('class').quantile([0.25, 0.75])[['l', 'w', 'h']]
# for class_ in dp['class'].unique()[2:4]:
#     Q1 = dp.loc[dp['class']==class_][['l', 'w', 'h']].quantile(0.25)
#     Q3 = dp.loc[dp['class']==class_][['l', 'w', 'h']].quantile(0.75)
#     IQR = Q3 - Q1

#     outlier = IQR * 1.5
#     min_outlier = Q1 - outlier
#     max_outlier = Q3 + outlier

In [159]:
dp.groupby('class').max()[['l', 'w', 'h']]

,l,w,h
class,,,
Bus,12.832408,3.367793,3.823763
Car,5.084260,2.181355,1.815112
Person,1.192694,1.097000,1.926561
SUV_&_Van,5.179612,2.243432,2.116742
Special_Vehicle,10.436841,3.283156,4.351732
Truck,10.408181,2.894754,3.771813
Two_Wheeler,2.695790,1.143500,1.822978


# # yolov5
---

In [162]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/yolov5_integ_2'

os.makedirs(f'{dst}/labels', exist_ok=True)
os.makedirs(f'{dst}/images', exist_ok=True)
os.makedirs(f'{dst}/ImageSets', exist_ok=True)
os.makedirs(f'{dst}/test_images', exist_ok=True)

class_num = {'Car': 0,
             'SUV_&_Van': 1,
             'Truck': 2,
             'Bus': 3,
             'Special_Vehicle': 4,
             'Two_Wheeler': 5,
             'Person': 6}

dat_typs = []

scenes = dp['scene'].unique()
for scene in tqdm(scenes):
    dat_typs.append(re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0])

    frames = dp.loc[dp['scene']==scene, 'frame'].unique()
    for frame in frames:


        # make points
        frame_data = dp.loc[(dp['scene']==scene) & (dp['frame']==frame)].copy()
        frame_data['class_num'] = frame_data['class'].apply(lambda x: class_num[x])
        xyxy_ls = frame_data[['min_x', 'min_y', 'max_x', 'max_y']].values
        
        xywhn_ls = []
        for xyxy in xyxy_ls:
            xywhn = xyxy2xywhn(xyxy)
            xywhn_ls.append(xywhn)
        
        frame_data[['xn', 'yn', 'wn', 'hn']] = xywhn_ls
        frame_data[['class_num', 'xn', 'yn', 'wn', 'hn']].to_csv(f'{dst}/labels/{scene}_{frame}.txt', header=False, index=False, sep=' ')


        # make images
        image_src = f'{src}/{scene}/camera/camera_0/{frame}.jpg'
        image_dst = f'{dst}/images/{scene}_{frame}.jpg'
        shutil.copyfile(image_src, image_dst)


# make ImageSets
train_ls = []
val_ls = []
test_ls = []

images = sorted(glob.glob(f'{dst}/images/*.jpg'))
# dat_typs = set([re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0] for scene in scenes])
for dat_typ in sorted(set(dat_typs)):
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.1, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    for j in train:
        for image in images:
            if j in image:
                train_ls.append(image)

    for j in val:
        for image in images:
            if j in image:
                val_ls.append(image)

    for j in test:
        for image in images:
            if j in image:
                test_ls.append(image)

with open(f'{dst}/ImageSets/train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(f'{dst}/ImageSets/val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(f'{dst}/ImageSets/test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))


# make test_images
with open(f'{dst}/ImageSets/test.txt', 'r') as f:
    test_images = [j.replace('\n', '') for j in f.readlines()]

for image in test_images:
    if os.path.exists(image) == True:
        shutil.move(image, f'{dst}/test_images')

100%|██████████| 4472/4472 [1:19:58<00:00,  1.07s/it]


# # pvrcnn
---

In [ ]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/pvrcnn_integ_2'

os.makedirs(f'{dst}/labels', exist_ok=True)
os.makedirs(f'{dst}/points', exist_ok=True)
os.makedirs(f'{dst}/ImageSets', exist_ok=True)

# # Z축 이동을 위해서 calib와 매칭하여 이동범위 지정
# calib_ls = []
# # scenes = []
# calibs = sorted(glob.glob(f'{src}/*/calib/camera/camera_0.json'))
# for calib in calibs:
#     scene = re.findall('[a-zA-Z0-9_]+', calib)[-5]
#     with open(calib, 'r') as f:
#         calib = json.load(f)
#     if calib['extrinsic'] not in calib_ls:
#         calib_ls.append(calib['extrinsic'])
#         # scenes.append(scene)    
# calib_typ = {'typ1': {'calib': calib_ls[0], 'mov_zpoint': 14},
#              'typ2': {'calib': calib_ls[1], 'mov_zpoint': 13},
#              'typ3': {'calib': calib_ls[2], 'mov_zpoint': 0},
#              'typ4': {'calib': calib_ls[3], 'mov_zpoint': -20}}

dat_typs = []

scenes = dp['scene'].unique()
for scene in tqdm(scenes):
    dat_typs.append(re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0])

    # with open(f'{src}/{scene}/calib/camera/camera_0.json', 'r') as f:
    #     calib = json.load(f)

    # for typ in ['typ1', 'typ2', 'typ3', 'typ4']:
    #     if calib['extrinsic'] == calib_typ[typ]['calib']:
    #         mov_zpoint = calib_typ[typ]['mov_zpoint']

    frames = dp.loc[dp['scene']==scene, 'frame'].unique()
    for frame in frames:


        # make labels
        frame_data = dp.loc[(dp['scene']==scene) & (dp['frame']==frame)].copy()
        frame_data[['point_x', 'point_y', 'point_z', 'l', 'w', 'h', 'rot_y', 'class']].to_csv(f'{dst}/labels/{scene}_{frame}.txt', header=False, index=False, sep=' ')


        # make points
        point_src = f'{src}/{scene}/lidar/{frame}.pcd'
        point_dst = f'{dst}/points/{scene}_{frame}.npy'

        pcd = o3d.t.io.read_point_cloud(point_src)
        positions = pcd.point.positions.numpy()
        intensity = pcd.point.intensity.numpy()
        positions[:, 2] += frame_data['mov_point_z'].values[0]

        pcd = np.concatenate((positions, intensity), axis = 1)
        # pcd 범위 자르기
        pcd = pcd[np.where((pcd[:, 0]>=-50) & (pcd[:, 0]<=50) & (pcd[:, 1]<=0) & (pcd[:, 1]>=-50) & (pcd[:, 2]>=-4) & (pcd[:, 2]<=8))]
        np.save(point_dst, pcd)


# make ImageSets
train_ls = []
val_ls = []
test_ls = []

points = sorted(glob.glob(f'{dst}/points/*.npy'))
for dat_typ in sorted(set(dat_typs)):
    # images_typ = [image for image in images if dat_typ in image]
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.1, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    for j in train:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                train_ls.append(point)

    for j in val:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                val_ls.append(point)

    for j in test:
        for point in points:
            if j in point:
                point = re.findall('[a-zA-Z0-9_]+', point)[-2]
                test_ls.append(point)

with open(f'{dst}/ImageSets/train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(f'{dst}/ImageSets/val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(f'{dst}/ImageSets/test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))

# # deepfusionmot
---

In [57]:
dp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424049 entries, 0 to 424048
Data columns (total 18 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   id           424049 non-null  int64  
 1   class        424049 non-null  object 
 2   min_x        424049 non-null  float64
 3   min_y        424049 non-null  float64
 4   max_x        424049 non-null  float64
 5   max_y        424049 non-null  float64
 6   point_x      424049 non-null  float64
 7   point_y      424049 non-null  float64
 8   point_z      424049 non-null  float64
 9   l            424049 non-null  float64
 10  w            424049 non-null  float64
 11  h            424049 non-null  float64
 12  rot_y        424049 non-null  float64
 13  intrinsic    424049 non-null  object 
 14  extrinsic    424049 non-null  object 
 15  mov_point_z  424049 non-null  int64  
 16  scene        424049 non-null  object 
 17  frame        424049 non-null  object 
dtypes: float64(11), int64(2)

In [59]:
src = '/data/NIA50/50-2/data/nia50_final/raw'
dst = '/data/NIA50/50-2/data/nia50_final/deepfusionmot_integ_2'

src_2d_result = '/data/NIA50/50-2/result/yolov5_integ_2/test_epoch11'
src_3d_result = '/data/NIA50/50-2/result/pvrcnn_integ_2/test_epoch7/eval'
src_scenes = '/data/NIA50/50-2/data/nia50_final/pvrcnn_integ_2/ImageSets/test.txt'

os.makedirs(f'{dst}/2D_yolov5', exist_ok=True)
os.makedirs(f'{dst}/3D_pvrcnn', exist_ok=True)
os.makedirs(f'{dst}/calib', exist_ok=True)
os.makedirs(f'{dst}/image_02', exist_ok=True)


# make 3D_pvrcnn dataframe
with open(f'{src_3d_result}/result.pkl', 'rb') as f:
    results = pkl.load(f)

frame_df_ls = []
for i, result in enumerate(tqdm(results, desc='3D_pvrcnn 데이터 프레임 생성 중')):
        
    scene_frame = result['frame_id']
    scene_ls = [scene_frame[:-5]] * len(result['name'])
    frame_ls = [scene_frame[-4:]] * len(result['name'])
    class_num_ls = result['pred_labels']
    scores = result['score']
    x_ls = result['boxes_lidar'][:, 0]
    y_ls = result['boxes_lidar'][:, 1]
    z_ls = result['boxes_lidar'][:, 2]
    l_ls = result['boxes_lidar'][:, 3]
    w_ls = result['boxes_lidar'][:, 4]
    h_ls = result['boxes_lidar'][:, 5]
    rot_y_ls = result['boxes_lidar'][:, 6]
    extrinsic = json.loads(dp.loc[dp['scene']==scene_ls[0], 'extrinsic'].values[0])
    extrinsic = np.asarray(extrinsic).reshape(4, 4)
    intrinsic = json.loads(dp.loc[dp['scene']==scene_ls[0], 'intrinsic'].values[0])
    intrinsic = np.asarray(intrinsic).reshape(3, 4)

    x1_ls, y1_ls, x2_ls, y2_ls = [], [], [], []
    for x, y, z, l, w, h, rot_y in zip(x_ls, y_ls, z_ls, l_ls, w_ls, h_ls, rot_y_ls):
        x1, y1, x2, y2 = xyz2xyxy(x, y, z, l, w, h, rot_y, extrinsic, intrinsic)
        x1_ls.append(x1)
        y1_ls.append(y1)
        x2_ls.append(x2)
        y2_ls.append(y2)

    data = {'scene': scene_ls,
            'frame': frame_ls, 
            'class_num': class_num_ls, 
            'x1': x1_ls, 
            'y1': y1_ls, 
            'x2': x2_ls, 
            'y2': y2_ls, 
            'score': scores, 
            'h': h_ls, 
            'w': w_ls, 
            'l': l_ls, 
            'x': x_ls, 
            'y': y_ls, 
            'z': z_ls, 
            'rot_y': rot_y_ls}
    globals()[f'frame_df{i}'] = pd.DataFrame(data=data)
    frame_df_ls.append(globals()[f'frame_df{i}'])
result_df = pd.concat(frame_df_ls)
result_df['frame'] = result_df['frame'].astype(int)
result_df['alpha'] = 0
result_df.loc[result_df['x1'] < 0, 'x1'] = 0
result_df.loc[result_df['x1'] > 1920, 'x1'] = 1920
result_df.loc[result_df['x2'] < 0, 'x2'] = 0
result_df.loc[result_df['x2'] > 1920, 'x2'] = 1920
result_df.loc[result_df['y1'] < 0, 'y1'] = 0
result_df.loc[result_df['y1'] > 1200, 'y1'] = 1200
result_df.loc[result_df['y2'] < 0, 'y2'] = 0
result_df.loc[result_df['y2'] > 1200, 'y2'] = 1200
result_df = result_df.loc[(result_df['x1']-result_df['x2']!=0) & (result_df['y1']-result_df['y2']!=0)]


# test scenes 불러오기
with open(src_scenes, 'r') as f:
    scenes = [re.sub('\n', '', i)[:-5] for i in f.readlines()]
    scenes = sorted(list(set(scenes)))
for scene in tqdm(scenes, desc='deepfusionmot 데이터 생성 중'):


    # make calib
    extrinsic = json.loads(dp.loc[dp['scene']==scene, 'extrinsic'].values[0])[:12]
    intrinsic = json.loads(dp.loc[dp['scene']==scene, 'intrinsic'].values[0])
    # kitti label 형태로 변환
    p2 = intrinsic
    R0_rect = np.eye(3).reshape(-1).tolist()
    Tr_velo_to_cam = extrinsic
    Tr_imu_to_velo = np.zeros((12)).tolist()

    calib_kitti =  ['P0: '+list2str(p2), 
                    'P1: '+list2str(p2), 
                    'P2: '+list2str(p2), 
                    'P3: '+list2str(p2), 
                    'R0_rect: '+list2str(R0_rect), 
                    'Tr_velo_to_cam: '+list2str(Tr_velo_to_cam), 
                    'Tr_imu_to_velo: '+list2str(Tr_imu_to_velo)]

    with open(f'{dst}/calib/{scene}.txt', 'w') as f:
        f.write('\n'.join(calib_kitti))


    # make 2D_yolov5
    w = 1920
    h = 1200
    labels = sorted(glob.glob(f'{src_2d_result}/labels/{scene}*.txt'))
    label_df = pd.DataFrame()
    for label in labels:
        # with open(label, 'r') as f:
        #     bbox = [re.sub('\n', '', j) for j in f.readlines()]
        frame_df = pd.read_csv(label, header=None, sep=' ')
        frame_df.columns = ['class', 'x', 'y', 'w', 'h', 'conf']
        frame_df['frame'] = int(re.findall('[0-9]+', label)[-1])
        frame_df['class'] += 1
        frame_df['xmin'] = (frame_df['x'] - frame_df['w']/2) * w
        frame_df['ymin'] = (frame_df['y'] - frame_df['h']/2) * h
        frame_df['xmax'] = (frame_df['x'] + frame_df['w']/2) * w
        frame_df['ymax'] = (frame_df['y'] + frame_df['h']/2) * h

        label_df = pd.concat((label_df, frame_df[['frame', 'class', 'xmin', 'ymin', 'xmax', 'ymax', 'conf']]), axis=0)
    label_df[['xmin', 'ymin', 'xmax', 'ymax']] = label_df[['xmin', 'ymin', 'xmax', 'ymax']].apply(lambda x: np.around(x))
    label_df.to_csv(f'{dst}/2D_yolov5/{scene}.txt', index=None, header=None, sep=',')


    # make 3D_pvrcnn
    result_df.loc[result_df['scene']==scene].iloc[:, 1:].to_csv(f'{dst}/3D_pvrcnn/{scene}.txt', index=None, header=None, sep=',')

    
    # make image_02
    img_src = f'{src}/{scene}/camera/camera_0'
    shutil.copytree(img_src, f'{dst}/image_02/{scene}', dirs_exist_ok=True)

deepfusionmot 데이터 생성 중: 100%|██████████| 476/476 [05:01<00:00,  1.58it/s]


# # TrackEval
---

In [11]:
src = '/data/NIA50/50-2/result/deepfusionmot_integ_2'
dst = '/data/NIA50/50-2/data/nia50_final/trackeval_integ_2_temp'

os.makedirs(f'{dst}/gt/label_02', exist_ok=True)
os.makedirs(f'{dst}/trackers/label_02', exist_ok=True)

scenes = sorted(os.listdir(f'{src}/image'))

for scene in scenes:

    # make gt, evaluate_tracking.seqmap.training
    seqmap = [f'{scene} empty 0000 0010' for scene in scenes]
    seqmap[0] = re.sub('0000', '0001', seqmap[0])
    with open(f'{dst}/gt/evaluate_tracking.seqmap.training', 'w') as f:
        f.write('\n'.join(seqmap))

    
    # make gt, label_02
    scene_df = dp.loc[dp['scene']==scene].copy()
    scene_df[['truncated', 'occluded', 'alpha']]= 0
    scene_df['frame'] = scene_df['frame'].astype(int)
    scene_df[['frame']]
    scene_df = scene_df[['frame', 'id', 'class', 'truncated', 'occluded', 'alpha', 'min_x', 'min_y', 'max_x', 'max_y', 'h', 'w', 'l', 'point_x', 'point_y', 'point_z', 'rot_y']]
    dropped_duple_idx = scene_df[['frame', 'id']].drop_duplicates().index
    scene_df = scene_df.loc[dropped_duple_idx].copy()
    scene_df.to_csv(f'{dst}/gt/label_02/{scene}.txt', index=None, header=None, sep=' ')


    # make trakers, label_02
    if os.path.isfile(f'{src}/data/{scene}.txt')==False:
        null_data = '0 0 None 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0'
        with open(f'{dst}/trackers/label_02/{scene}.txt', 'w') as f:
            f.write(null_data)
    else:
        shutil.copy(f'{src}/data/{scene}.txt', f'{dst}/trackers/label_02/{scene}.txt')

In [16]:
allcat = glob.glob('/data/NIA50/50-2/result/trackeval_integ_2/label_02/*.txt')

allcat_ls = []
for cat in allcat:
    cat_name = re.findall('[a-z_&]+_summary', cat)[0]
    cat_name = re.sub('_summary', '', cat_name)
    cat_df = pd.read_csv(cat, sep=' ')
    cat_df.index = [cat_name]

    allcat_ls.append(cat_df)

allcat_df = pd.concat(allcat_ls, axis=0)
allcat_df = allcat_df.iloc[1:, :][['HOTA', 'MOTA', 
                                    'DetA', 'AssA', 'DetRe', 'DetPr', 'LocA', 'OWTA',
                                    'MOTP', 'MODA',
                                    'sMOTA', 'IDF1', 'IDR', 'IDP', 'IDTP', 'IDFN', 'IDFP', 'Dets', 'GT_Dets', 'IDs', 'GT_IDs']]
allcat_df.to_csv(f'/data/NIA50/50-2/result/trackeval_integ_2/allcat_summary.csv')

In [17]:
allcat_df.iloc[1:, :][['HOTA', 'MOTA', 
                       'DetA', 'AssA', 'DetRe', 'DetPr', 'LocA', 'OWTA',
                       'MOTP', 'MODA',
                       'sMOTA', 'IDF1', 'IDR', 'IDP', 'IDTP', 'IDFN', 'IDFP', 'Dets', 'GT_Dets', 'IDs', 'GT_IDs']]

,HOTA,MOTA,DetA,AssA,DetRe,DetPr,LocA,OWTA,MOTP,MODA,...,IDF1,IDR,IDP,IDTP,IDFN,IDFP,Dets,GT_Dets,IDs,GT_IDs
two_wheeler,64.862,26.908,48.067,88.252,76.694,53.946,89.041,82.075,88.144,28.012,...,69.320,83.936,59.040,1672,320,1160,2832,1992,588,269
bus,60.885,32.737,42.388,87.753,51.191,69.741,93.209,66.968,93.064,33.760,...,60.472,52.430,71.429,410,372,164,574,782,122,184
suv_&_van,63.876,36.891,48.943,83.844,62.512,68.017,93.023,72.287,93.088,38.324,...,66.231,63.550,69.147,8205,4706,3661,11866,12911,2181,1856
special_vehicle,34.584,-162.920,13.805,87.072,43.191,16.625,87.742,61.207,86.431,-162.140,...,26.705,48.042,18.492,184,199,811,995,383,232,50
car,65.996,42.191,50.556,86.756,62.656,70.545,92.356,73.587,92.035,43.819,...,68.283,64.465,72.582,11206,6177,4233,15439,17383,2637,2651
truck,44.395,-30.327,23.880,83.107,41.168,35.726,91.116,58.347,92.308,-29.913,...,39.128,42.108,36.542,915,1258,1589,2504,2173,568,331


In [15]:
allcat_df.iloc[1:, :][['HOTA', 'MOTA', 
                       'DetA', 'AssA', 'DetRe', 'DetPr', 'LocA', 'OWTA',
                       'MOTP', 'MODA',
                       'sMOTA', 'IDF1', 'IDR', 'IDP', 'IDTP', 'IDFN', 'IDFP', 'Dets', 'GT_Dets', 'IDs', 'GT_IDs']]

,HOTA,MOTA,DetA,AssA,DetRe,DetPr,LocA,OWTA,MOTP,MODA,...,IDF1,IDR,IDP,IDTP,IDFN,IDFP,Dets,GT_Dets,IDs,GT_IDs
two_wheeler,59.317,25.8930,41.7030,85.153,59.023,56.021,88.812,70.685,87.939,25.893,...,63.913,65.625,62.288,147,77,89,236,224,29,35
bus,20.854,-145.5800,8.1101,53.746,21.697,11.391,89.894,34.117,90.311,-144.220,...,14.520,21.088,11.071,31,116,249,280,147,50,43
suv_&_van,71.041,56.1710,57.5190,88.624,67.494,76.818,91.560,77.119,91.010,56.882,...,76.237,71.610,81.503,1410,559,320,1730,1969,242,289
special_vehicle,27.825,14.7830,18.8590,41.366,20.229,70.494,91.101,28.831,90.665,14.783,...,33.784,21.739,75.758,25,90,8,33,115,11,16
car,70.086,55.3460,57.3480,86.430,65.612,79.454,92.036,75.121,91.640,56.918,...,74.302,67.830,82.140,2157,1023,469,2626,3180,396,539
truck,49.728,3.2653,31.3630,79.152,45.800,48.787,91.443,60.145,90.776,4.898,...,49.263,47.755,50.870,117,128,113,230,245,46,53
